In [2]:
import csv
import json

def extract_fields(properties, prefix='', fields=None):
    if fields is None:
        fields = []

    for field, details in properties.items():
        field_path = f"{prefix}.{field}" if prefix else field

        # Handle the extraction of field type and additional settings
        field_info = {
            'field_path': field_path,
            'type': details.get('type', 'n/a'),
            'additional_settings': json.dumps({
                k: v for k, v in details.items() if k not in ['type', 'properties', 'fields']
            }),
        }
        fields.append(field_info)

        # Handle multi-fields like "keyword"
        if 'fields' in details:
            for sub_field, sub_details in details['fields'].items():
                sub_field_path = f"{field_path}.{sub_field}"
                sub_field_info = {
                    'field_path': sub_field_path,
                    'type': sub_details['type'],
                    'additional_settings': json.dumps({
                        k: v for k, v in sub_details.items() if k != 'type'
                    }),
                }
                fields.append(sub_field_info)

        # Recurse into nested properties
        if 'properties' in details:
            extract_fields(details['properties'], prefix=field_path, fields=fields)

    return fields


def write_to_csv(fields, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['field_path', 'type', 'additional_settings']  # Updated fieldnames
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for field in fields:
            writer.writerow(field)


def main():
    # Load your Elasticsearch index mapping from JSON file or Elasticsearch API response
    mapping_json = '../ta-main-index-mappings-old.json'  # Replace with your file path or JSON content
    
    with open(mapping_json, 'r') as file:
        mapping = json.load(file)
    
    # Extract fields assuming the mapping is wrapped in a "mappings" key
    mappings = mapping.get('mappings', {})
    properties = mappings.get('properties', {})
    
    fields = extract_fields(properties)

    # Write the fields information to CSV
    write_to_csv(fields, 'gpt-tps-vdx-go-es_mapping_details.csv')

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'scripts/ta-main-index-mappings-old.json'